In [ ]:
import pandas as pd

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import confusion_matrix

In [ ]:
split_date = '2023-05-01'

In [ ]:
li = []

li.append(pd.read_csv("../prepared_data/test.csv", index_col=0))

matches = pd.concat(li, axis=0, ignore_index=True)
matches = matches[matches["venue"] == "Home"]
matches.shape

(6324, 125)

In [ ]:
rf3 = RandomForestClassifier(n_estimators=10000, n_jobs=-1, random_state=1, max_depth=3)
rf4 = RandomForestClassifier(n_estimators=10000, n_jobs=-1, random_state=1, max_depth=3)

In [ ]:
train = matches[matches["date"] < split_date]
test = matches[matches["date"] > split_date]

In [ ]:
predictors3 = ["gf_rolling", "ga_rolling", "sh_rolling", "sot%_rolling", "dist_rolling"]
predictors4 = ["team_code", "opponent_code"]

In [ ]:
rf3.fit(train[predictors3], train["target"])

RandomForestClassifier(max_depth=3, n_estimators=10000, n_jobs=-1,
                       random_state=1)

In [ ]:
predsWinsTrain3 = rf3.predict(train[predictors3])

probabilities3 = rf3.predict_proba(test[predictors3])
probabilities3 = probabilities3[:, 1]

combinedProbabilities3 = pd.DataFrame(dict(actual=test["target"], probabilities=probabilities3), index=test.index)
combinedProbabilities3["date"] = test["date"]

#### Finding the right treshhold

In [ ]:
best_threshold = 0
min_diff = float('inf')

thresholds = range(0, 1000, 1)
for threshold in thresholds:
	threshold = threshold / 1000
	predicted = (probabilities >= threshold).astype('int')
	tn, fp, fn, tp = confusion_matrix(combinedProbabilities["actual"], predicted).ravel()
        
	pos = tp + fp
	neg = tn + fn
	diff = abs(neg - (100 / 37.8) * pos)
	
	if diff < min_diff:
		min_diff = diff
		best_threshold = threshold

print(best_threshold)
predicted = (probabilities >= best_threshold).astype('int')
combinedProbabilities["predicted"] = predicted

0.477


In [ ]:
print(int(len(combinedProbabilities[(combinedProbabilities["actual"] == 1) & (combinedProbabilities["predicted"] == 1)]) / len(combinedProbabilities[combinedProbabilities["actual"] == 1]) * 1_000_000) / 10_000, "The higher the better, but doesn't have to be high")
print(int(len(combinedProbabilities[(combinedProbabilities["actual"] == 0) & (combinedProbabilities["predicted"] == 1)]) / len(combinedProbabilities[combinedProbabilities["actual"] == 0]) * 1_000_000) / 10_000, "The smaller the better, has to be small")

print(int(len(combinedProbabilities[(combinedProbabilities["actual"] == 1) & (combinedProbabilities["predicted"] == 1)]) / len(combinedProbabilities[combinedProbabilities["predicted"] == 1]) * 1_000_000) / 10_000, "Overall measure of performance in %")

36.1746 The higher the better, but doesn't have to be high
19.0566 The smaller the better, has to be small
63.2727 Overall measure of performance in %


In [ ]:
pd.DataFrame(confusion_matrix(combinedProbabilities["actual"], combinedProbabilities["predicted"]))

,0,1
0,429,101
1,307,174


In [ ]:
combinedProbabilities.sort_values("date").tail(50)

,actual,probabilities,date,predicted
5168,1,0.492517,2024-01-21,1
9757,0,0.337247,2024-01-21,0
4111,1,0.387008,2024-01-21,0
4724,1,0.316091,2024-01-21,0
10431,0,0.396134,2024-01-21,0
9299,1,0.560583,2024-01-21,1
2245,0,0.579910,2024-01-21,1
8787,1,0.414899,2024-01-21,0
1906,0,0.519641,2024-01-21,1
1706,0,0.571284,2024-01-21,1
